<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
from plan import plan; plan("types", "regexps")

### expressions régulières

* notion transverse aux langages de programmation
* présente dans la plupart d'entre eux
* en particulier historiquement Perl  
  qui en avait fait un *first-class citizen* 

### exemples

* `a*` pour reconnaître tous les mots composés de 0 ou plusieurs `a`
  * `''`, `'a'`, `'aa'`, … sont les mots qui *matchent*
* `(ab)+` : toutes les suites de au moins 1 occurrence de `ab`  
  * `'ab'`, `'abab'`, `'ababab'`, … sont les mots qui *matchent*

### le module `re`

en Python, les expressions régulières sont accessibles au travers du module `re`

In [ ]:
import re

# en anglais on dit pattern
# en français on dit filtre, 
# ou encore parfois motif
pattern = "a*"

# la fonction `match` 
re.match(pattern, '')

In [ ]:
re.match(pattern, 'a')

In [ ]:
re.match(pattern, 'aa')

In [ ]:
re.match('(ab)+', 'ab')

In [ ]:
# retourne None
re.match('(ab)+', 'ba')

### `re.match()` 

* **ATTENTION** car `re.match()` vérifie si l'expression régulière peut être trouvée **au début** de la chaine

In [ ]:
match = re.match('(ab)+', 'ababzzz')
match

In [ ]:
match.start()

In [ ]:
match.end()

### les objets `Match` 

* le résultat de `re.match()` est de type `Match` 
* pour les détails de ce qui a été trouvé
* par exemple quelle partie de la chaine
* et aussi les sous-chaines correspondant aux groupes  
  (on en reparlera)


### variantes 

* `re.search()` pour chercher le pattern n'importe où dans la chaine
* `re.findall()` et `re.finditer()` pour trouver toutes les occurences du filtre dans la chaine
* `re.sub()` pour remplacer …

**notre sujet**

* ici nous nous intéressons surtout à la façon de **construire les regexps**
* se reporter à la documentation du module pour ces variantes

### pour visualiser

In [ ]:
# digression : un utilitaire pour montrer
# le comportement d'un pattern / filtre

def match_all(pattern, strings):
    """
    match a pattern agains a set of strings and shows result
    """
    margin = max(len(x) for x in strings) + 2 # for the quotes
    for string in strings:
        print(f"`{pattern}` ⇆ {'`'+string+'`':>{margin}} → ", end="")
        match = re.match(pattern, string)
        if not match:
            print("NO")
        elif not (match.start() == 0 and match.end() == len(string)):
            # start() is always 0
            print(f"NO (yes until {match.end()})")
        else:
            print("YES")

In [ ]:
match_all('(ab)+', ['ab', 'abab', 'ababzzz', ''])

### filtrer **un seul** caractère : `[..]`

* avec les `[]` on peut désigner un **ensemble** de caractères :
* `[a-z]` les lettres minuscules
* `[a-zA-Z0-9_]` les lettres et chiffres et underscore

In [ ]:
match_all('[a-z]', ['a', '', '0'])

In [ ]:
match_all('[a-z0-9]', ['a', '9', '-'])

In [ ]:
# poubn insérer un '-', le mettre à la fin
match_all('[0-9+-]', ['0', '+', '-', 'A'])

### n'importe quel caractère : `.`

In [ ]:
match_all('.', ['', 'a', '.', 'Θ', 'ab'])

### idem mais à l'envers : `[^..]`

* si l'ensemble de caractères entre `[]` commence par un `^`
* cela désigne le **complémentaire** dans l'espace des caractères

In [ ]:
# complémentaires
match_all('[^a-z]', ['a', '0', '↑', 'Θ'])

In [ ]:
match_all('[^a-z0-9]', ['a', '9', '-'])

### 0 ou plusieurs occurrences : `..*`

In [ ]:
match_all('[a-z]*', ['', 'cba', 'xyz9'])

In [ ]:
match_all('(ab)*', ['', 'ab', 'abab'])

### 1 ou plusieurs occurrences : `..+`

In [ ]:
match_all('[a-z]+', ['', 'cba', 'xyz9'])

In [ ]:
match_all('(ab)+', ['', 'ab', 'abab'])

### concaténation

quand on concatène deux filtres, la chaine doit matcher l'un puis l'autre

In [ ]:
# c'est le seul mot qui matche
match_all('ABC', ['ABC']) 

In [ ]:
match_all('A*B', ['B', 'AB', 'AAB', 'AAAB']) 

### groupement : `(..)`

* comme déjà vu avec `(ab)+`
  * permet d'appliquer un opérateur sur une regexp
* cela définit un **groupe** qui peut être retrouvé dans le match
  * grâce à la méthode `groups()` 

In [ ]:
# groupes anonymes
pattern = "([a-z]+)=([a-z0-9]+)"

string = "foo=barbar99"

match = re.match(pattern, string)
match

In [ ]:
# dans l'ordre où ils apparaissent
match.groups()

### alternative : `..|..`

pour filtrer avec une regexp **ou** une autre :

In [ ]:
match_all('ab|cd', ['ab', 'cd', 'abcd'])

In [ ]:
match_all('ab|cd*', ['ab', 'c', 'cd', 'cdd'])

In [ ]:
match_all('ab|(cd)*', ['ab', 'c', 'cd', 'cdd'])

In [ ]:
match_all('(ab|cd)*', ['ab', 'c', 'cd', 'cdd', 'abcd'])

### 0 ou 1 occurrences : `..?`

In [ ]:
match_all('[a-z]?', ['', 'b', 'xy'])

### nombre d'occurrences dans un intervalle : `..{n,m}`

* `a{3}` : exactement 3 occurrences de `a`
* `a{3,}` : au moins 3 occurrences
* `a{3,6}` : entre 3 et 6 occurrences

In [ ]:
match_all('(ab){1,3}', ['', 'ab', 'abab', 'ababab', 'ababababababab'])

### classes de caractères

raccourcis qui filtrent **un caractère** dans une classe  
définis en fonction de la configuration de l'OS en termes de langue

* `\s` (pour Space) : exactement un caractère de séparation (typiquement Espace, Tabulation, Newline)
* `\w` (pour Word) : exactement un caractère alphabétique ou numérique
* `\d` (pour Digit) : un chiffre
* `\S`, `\W` et `\D` : les complémentaires

In [ ]:
match_all('\w+', ['eFç0', 'été', ' ta98'])

In [ ]:
match_all('\s?\w+', ['eFç0', 'été', ' ta98'])

### groupe nommé : `(?P<name>..)`

* le même effet que les groupes anonymes,
* mais on peut retrouver le contenu depuis le nom du groupe
* plutôt que le rang du groupe
* qui peut rapidement devenir une notion fragile / peu maintenable

In [ ]:
# groupes nommés
pattern = "(?P<variable>[a-z]+)=(?P<valeur>[a-z0-9]+)"

string = "foo=barbar99"

match = re.match(pattern, string)
match

In [ ]:
match.group('variable')

In [ ]:
match.group('valeur')

### plusieurs occurrences d'un groupe : `(?P=name)`

on peut spécifier qu'un groupe apparaisse plusieurs fois

In [ ]:
# la deuxième occurrence de <nom> doit être la même que la première
pattern = '(?P<nom>\w+).*(?P=nom)'

string1 = 'Jean again Jean'
string2 = 'Jean nope Pierre'

match_all(pattern, [string1, string2])

### pour aller plus loin

* testeurs en ligne  
  <https://pythex.org>  
  <https://regex101.com/> (bien choisir Python)

* un peu de détente, avec ce jeu de mots croisés basé sur les regexps 
  <https://regexcrossword.com>

* tour complet de la syntaxe des regexps  
  <https://docs.python.org/3/library/re.html#regular-expression-syntax>